## ***Jun 8 2023***

Энергия Kch для одного захода без точки пучка в фите и с ней:
<div>
<img src="PictureDump/KchEnergy_NoBeamPoint_run61683.png" alt="Drawing" style="width: 550px;"/>
<img src="PictureDump/KchEnergy_WithBeamPoint_run61683.png" alt="Drawing" style="width: 550px;"/>
</div>

Импульс Kch для одного захода без точки пучка в фите и с ней:
<div>
<img src="PictureDump/KchMom_NoBeamPoint_run61683.png" alt="Drawing" style="width: 550px;"/>
<img src="PictureDump/KchMom_WithBeamPoint_run61683.png" alt="Drawing" style="width: 550px;"/>
</div>